<a href="https://colab.research.google.com/github/nelsonbeas33/Datos-Masivos/blob/main/tarea5/tarea5_datosMasivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prueba de analisis de datos con spark, haremos transformaciones de prueba para el proyecto

In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install langchain-huggingface

import os
import sys


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,513 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,224 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRe

In [3]:
# Crear la sesión de Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("IrisDataset").getOrCreate()

# Importar el dataset Iris desde sklearn y convertirlo a un DataFrame de PySpark
from sklearn.datasets import load_iris
import pandas as pd

# Cargar el dataset Iris desde sklearn
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target

# Crear un DataFrame de PySpark
df = spark.createDataFrame(iris_df)

# Mostrar las primeras filas y el esquema
df.show(5)
df.printSchema()


+-----------------+----------------+-----------------+----------------+-------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|species|
+-----------------+----------------+-----------------+----------------+-------+
|              5.1|             3.5|              1.4|             0.2|      0|
|              4.9|             3.0|              1.4|             0.2|      0|
|              4.7|             3.2|              1.3|             0.2|      0|
|              4.6|             3.1|              1.5|             0.2|      0|
|              5.0|             3.6|              1.4|             0.2|      0|
+-----------------+----------------+-----------------+----------------+-------+
only showing top 5 rows

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- species: long (nullable = true)



In [4]:
from pyspark.ml.feature import VectorAssembler

# Seleccionar columnas de características
feature_columns = iris.feature_names

# Crear una columna de características
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_transformed = assembler.transform(df)

# Mostrar DataFrame transformado
df_transformed.select("features", "species").show(5, truncate=False)


+-----------------+-------+
|features         |species|
+-----------------+-------+
|[5.1,3.5,1.4,0.2]|0      |
|[4.9,3.0,1.4,0.2]|0      |
|[4.7,3.2,1.3,0.2]|0      |
|[4.6,3.1,1.5,0.2]|0      |
|[5.0,3.6,1.4,0.2]|0      |
+-----------------+-------+
only showing top 5 rows



In [5]:
# Dividir el dataset en entrenamiento (80%) y prueba (20%)
train_data, test_data = df_transformed.randomSplit([0.8, 0.2], seed=42)


In [6]:
from pyspark.ml.classification import RandomForestClassifier

# Crear y entrenar el modelo
rf = RandomForestClassifier(labelCol="species", featuresCol="features", numTrees=100)
model = rf.fit(train_data)


In [7]:
# Hacer predicciones en los datos de prueba
predictions = model.transform(test_data)

# Mostrar algunas predicciones
predictions.select("features", "species", "prediction").show(5)


+-----------------+-------+----------+
|         features|species|prediction|
+-----------------+-------+----------+
|[4.4,3.0,1.3,0.2]|      0|       0.0|
|[4.6,3.2,1.4,0.2]|      0|       0.0|
|[4.6,3.6,1.0,0.2]|      0|       0.0|
|[4.8,3.1,1.6,0.2]|      0|       0.0|
|[4.9,3.1,1.5,0.2]|      0|       0.0|
+-----------------+-------+----------+
only showing top 5 rows



In [8]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluar precisión del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="species", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Precisión del modelo: {accuracy:.2f}")


Precisión del modelo: 0.97


In [9]:
# Crear una matriz de confusión
predictions.groupBy("species", "prediction").count().orderBy("species", "prediction").show()


+-------+----------+-----+
|species|prediction|count|
+-------+----------+-----+
|      0|       0.0|   13|
|      1|       1.0|    6|
|      2|       1.0|    1|
|      2|       2.0|   12|
+-------+----------+-----+

